In [26]:
import pandas as pd

# Load the training data
train_data = pd.read_csv('train.csv')

# Display the first few rows of the dataset
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [27]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder

# Impute missing values
imputer = SimpleImputer(strategy='median')
train_data['Age'] = imputer.fit_transform(train_data[['Age']])

# Fill missing 'Embarked' with the mode
mode_embarked = train_data['Embarked'].mode()[0]
train_data['Embarked'].fillna(mode_embarked, inplace=True)

# Drop 'Cabin' as an example of simplification (or transform it)
train_data.drop(columns=['Cabin'], inplace=True)

# Encode categorical variables
label_encoder = LabelEncoder()
train_data['Sex'] = label_encoder.fit_transform(train_data['Sex'])
train_data['Embarked'] = label_encoder.fit_transform(train_data['Embarked'])

# Drop unnecessary features
train_data.drop(columns=['PassengerId', 'Name', 'Ticket'], inplace=True)

# Display the cleaned data
train_data.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [28]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the data into features and target
X = train_data.drop('Survived', axis=1)
y = train_data['Survived']

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Logistic Regression model
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Predict on validation set
y_pred = logistic_model.predict(X_val)

# Calculate and print the accuracy
accuracy = accuracy_score(y_val, y_pred)
accuracy


0.8100558659217877

In [29]:
from sklearn.tree import DecisionTreeClassifier

# Initialize and train the Decision Tree model
decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X_train, y_train)

# Predict on validation set
y_pred_dt = decision_tree_model.predict(X_val)

# Calculate and print the accuracy
accuracy_dt = accuracy_score(y_val, y_pred_dt)
accuracy_dt


0.776536312849162

In [30]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the Random Forest model
random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_model.fit(X_train, y_train)

# Predict on validation set
y_pred_rf = random_forest_model.predict(X_val)

# Calculate and print the accuracy
accuracy_rf = accuracy_score(y_val, y_pred_rf)
accuracy_rf


0.8212290502793296

In [31]:
from sklearn.svm import SVC

# Initialize and train the SVM model
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

# Predict on validation set
y_pred_svm = svm_model.predict(X_val)

# Calculate and print the accuracy
accuracy_svm = accuracy_score(y_val, y_pred_svm)
accuracy_svm


0.7821229050279329

In [32]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize and train the KNN model
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

# Predict on validation set
y_pred_knn = knn_model.predict(X_val)

# Calculate and print the accuracy
accuracy_knn = accuracy_score(y_val, y_pred_knn)
accuracy_knn


0.7206703910614525

In [33]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

# Define categorical and numerical features
categorical_features = ['Sex', 'Embarked']
numerical_features = ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass']

# Create transformers for the pipeline
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Combine transformers into a preprocessor with ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a full pipeline with the preprocessor and a classifier
logistic_regression_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

# Fit the pipeline on the training data
logistic_regression_pipeline.fit(X_train, y_train)

# Predict on validation set
y_pred_pipeline = logistic_regression_pipeline.predict(X_val)

# Calculate and print the accuracy
accuracy_pipeline = accuracy_score(y_val, y_pred_pipeline)
accuracy_pipeline


0.8100558659217877

In [34]:
from sklearn.tree import DecisionTreeClassifier

# Create a full pipeline with the preprocessor and a classifier
decision_tree_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Fit the pipeline on the training data
decision_tree_pipeline.fit(X_train, y_train)

# Predict on validation set
y_pred_dt_pipeline = decision_tree_pipeline.predict(X_val)

# Calculate and print the accuracy
accuracy_dt_pipeline = accuracy_score(y_val, y_pred_dt_pipeline)
accuracy_dt_pipeline


0.776536312849162

In [35]:
from sklearn.ensemble import RandomForestClassifier

# Create a full pipeline with the preprocessor and a classifier
random_forest_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Fit the pipeline on the training data
random_forest_pipeline.fit(X_train, y_train)

# Predict on validation set
y_pred_rf_pipeline = random_forest_pipeline.predict(X_val)

# Calculate and print the accuracy
accuracy_rf_pipeline = accuracy_score(y_val, y_pred_rf_pipeline)
accuracy_rf_pipeline


0.8212290502793296